In [ ]:
# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(data)

# Reshape data to have the format (samples, time_steps, features)
data_reshaped = data_normalized.reshape(
    data_normalized.shape[0], data_normalized.shape[1], 1)

# Generate labels for the data
labels = [i // 2 for i in range(12)] * (data.shape[0] // 12)


In [ ]:
# Function to process data
def process_data(data):
    # Apply STFT with 30ms windows using Hamming window
    f, t, Zxx = stft(data, fs=100000, window='hamming', nperseg=3000)

    # Perform FFT
    fft_data = np.fft.fft(Zxx)

    # Square FFT values
    energy_magnitude = np.abs(fft_data) ** 2

    return energy_magnitude


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

# Build the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# Create a checkpoint to save the best model
checkpoint = ModelCheckpoint(
    "best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(
    X_test, y_test), callbacks=[checkpoint], shuffle=True)


In [ ]:

# Load the best model
best_model = tf.keras.models.load_model("best_model.h5")

# Evaluate the model on the test set
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

# Make predictions
predictions = best_model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
```


In [ ]:
# Convert the Keras model to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
